<a href="https://colab.research.google.com/github/niranjanbsubramanian/save-model-keras/blob/main/save%20model%20in%20keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.datasets import fashion_mnist
from keras.utils import np_utils
 
batch_size = 128
num_classes = 10
epochs = 50
 
# input image dimensions
img_rows, img_cols = 28, 28
 
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
 
 
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
 
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
 
# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)
 
#Building our CNN
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
 
#compile the model
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [12]:
history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=0.2,
          )
 
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/50
375/375 [==============================] - 4s 9ms/step - loss: 2.2843 - accuracy: 0.1115 - val_loss: 2.2287 - val_accuracy: 0.2013
Epoch 2/50
375/375 [==============================] - 3s 8ms/step - loss: 2.2258 - accuracy: 0.1721 - val_loss: 2.1580 - val_accuracy: 0.3706
Epoch 3/50
375/375 [==============================] - 3s 8ms/step - loss: 2.1575 - accuracy: 0.2557 - val_loss: 2.0730 - val_accuracy: 0.4545
Epoch 4/50
375/375 [==============================] - 3s 9ms/step - loss: 2.0786 - accuracy: 0.3399 - val_loss: 1.9675 - val_accuracy: 0.5261
Epoch 5/50
375/375 [==============================] - 3s 9ms/step - loss: 1.9763 - accuracy: 0.4051 - val_loss: 1.8390 - val_accuracy: 0.5569
Epoch 6/50
375/375 [==============================] - 3s 8ms/step - loss: 1.8601 - accuracy: 0.4421 - val_loss: 1.6968 - val_accuracy: 0.5731
Epoch 7/50
375/375 [==============================] - 3s 8ms/step - loss: 1.7347 - accuracy: 0.4641 - val_loss: 1.5541 - val_accuracy: 0.5906
Epoch 

In [13]:
model.save('Fashion_Mnist_CNN.h5')

In [14]:
from keras.models import load_model
restored_model = load_model('/content/Fashion_Mnist_CNN.h5')
#restored_model.evaluate(x_test, y_test, verbose=1)

In [15]:
import numpy as np
index = np.random.randint(0, len(x_test)) # randomly choose a sample from the test set
sample = np.array([x_test[index]])
prediction = restored_model.predict(sample)
print('Sample Number:',index,'\n','\nPredicted Class:',np.argmax([prediction[0]]), 'Actual Class:',np.argmax(y_test[index]))

Sample Number: 9990 
 
Predicted Class: 5 Actual Class: 5


### Saving a model to .json

In [16]:
json_model = model.to_json()
with open('model.json','w') as json_file:
  json_file.write(json_model)

In [17]:
#saving model weights
model.save_weights('weights.h5')

### Load Json Model

In [23]:
from keras.models import model_from_json

with open('/content/model.json', 'r') as json_file:
  json_string = json_file.read()
json_file.close()
json_model = model_from_json(json_string)
json_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)              

#### Load weights to json model

In [24]:
json_model.load_weights('/content/weights.h5')

In [25]:
json_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
#json_model.evaluate(x_test, y_test, verbose=1)

### JSON Model Prediction

In [26]:
import numpy as np
index = np.random.randint(0, len(x_test)) # Get a random index
sample = np.array([x_test[index]]) # select a random sample
prediction = json_model.predict(sample) # predict the sample
print('Sample Number:',index,'\n','\nPredicted Class:',np.argmax([prediction[0]]), 'Actual Class:',np.argmax(y_test[index]))

Sample Number: 4703 
 
Predicted Class: 5 Actual Class: 5


### Save and load the model in YAML format

In [27]:
yaml_model = model.to_yaml()
with open('model.yaml','w') as yaml_file:
  yaml_file.write(yaml_model)

In [28]:
from keras.models import model_from_yaml

# model reconstruction from yaml
with open('/content/model.yaml','r') as yaml_file:
  yaml_string = yaml_file.read()
yaml_model = model_from_yaml(yaml_string)
yaml_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)              

In [29]:
yaml_model.load_weights('/content/weights.h5')

In [30]:
yaml_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
#yaml_model.evaluate(x_test, y_test, verbose=1)

### YAML Model Prediction

In [31]:
import numpy as np
index = np.random.randint(0, len(x_test)-1)
sample = np.array([x_test[index]])
prediction = yaml_model.predict(sample)
print('Sample Number:',index,'\n','\nPredicted Class:',np.argmax([prediction[0]]), 'Actual Class:',np.argmax(y_test[index]))

Sample Number: 5408 
 
Predicted Class: 9 Actual Class: 9


### Model CheckPoint Callback

In [32]:
# import the callback
from keras.callbacks import ModelCheckpoint

# create the callback
filepath = '/content/weight/weights-{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_ckpt = ModelCheckpoint(monitor='val_loss', save_best_only=True, verbose=1, mode='auto',filepath=filepath)

# pass the callback to the fit() method
history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=0.2,
          callbacks=model_ckpt)

Epoch 1/50
375/375 [==============================] - 4s 9ms/step - loss: 0.7767 - accuracy: 0.7259 - val_loss: 0.6343 - val_accuracy: 0.7809

Epoch 00001: val_loss improved from inf to 0.63431, saving model to /content/weight/weights-01-0.78.hdf5
Epoch 2/50
375/375 [==============================] - 3s 9ms/step - loss: 0.7706 - accuracy: 0.7304 - val_loss: 0.6311 - val_accuracy: 0.7816

Epoch 00002: val_loss improved from 0.63431 to 0.63106, saving model to /content/weight/weights-02-0.78.hdf5
Epoch 3/50
375/375 [==============================] - 3s 9ms/step - loss: 0.7640 - accuracy: 0.7326 - val_loss: 0.6267 - val_accuracy: 0.7818

Epoch 00003: val_loss improved from 0.63106 to 0.62669, saving model to /content/weight/weights-03-0.78.hdf5
Epoch 4/50
375/375 [==============================] - 3s 8ms/step - loss: 0.7595 - accuracy: 0.7328 - val_loss: 0.6239 - val_accuracy: 0.7830

Epoch 00004: val_loss improved from 0.62669 to 0.62387, saving model to /content/weight/weights-04-0.78.h